In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [6]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {letter:num for num,letter in enumerate(world_set)}

In [7]:
print(vocab)

{'.': 0, 'g': 1, 't': 2, 'm': 3, 'r': 4, 'p': 5, 'i': 6, 'o': 7, "'": 8, ',': 9, 'e': 10, 'd': 11, 'h': 12, 'w': 13, 'k': 14, 'f': 15, 'B': 16, 'u': 17, 's': 18, 'n': 19, 'y': 20, 'c': 21, ' ': 22, 'b': 23, 'a': 24, 'l': 25}


In [8]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [9]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [12]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+ sequence_length]
  y_str = sentence[i+1:i+ 1 + sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [13]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[16, 4, 6, 21, 14, 22, 13, 24, 25, 25]
[4, 6, 21, 14, 22, 13, 24, 25, 25, 18]


In [14]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(vocab_size)[x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [15]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [16]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0.,

In [17]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 4,  6, 21, 14, 22, 13, 24, 25, 25, 18])


In [18]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, layers,batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)
  
  def forward(self, x):
    x,xx = self.rnn(x)
    x = self.fc(x)
    ##
    ##
    return x

In [19]:
net = Net(vocab_size, hidden_size, 2)

In [20]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [21]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [22]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

kh  kkk ku khu  hskhk h  khkh  kh uk h B skhs uk   kh  sukBskshu kkk    kk ku khu  kBkBskhk h kskkk s kh uskk  whk skuk  uB   ku kkh ksku kkk    kk ku khu  hskhk h kh   ks ks  kk hs khskh  u us su 
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [23]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [28]:
sentence = ("Do you want to build a snowman"
          "Come on let's go and play"
          "I never see you anymore Come out the door"
          "It's like you've gone away")

In [29]:
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {letter:num for num,letter in enumerate(world_set)}

In [30]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 25


In [31]:
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 6  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [32]:
# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+ sequence_length]
  y_str = sentence[i+1:i+ 1 + sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Do you -> o you 
1 o you  ->  you w
2  you w -> you wa
3 you wa -> ou wan
4 ou wan -> u want
5 u want ->  want 
6  want  -> want t
7 want t -> ant to
8 ant to -> nt to 
9 nt to  -> t to b
10 t to b ->  to bu
11  to bu -> to bui
12 to bui -> o buil
13 o buil ->  build
14  build -> build 
15 build  -> uild a
16 uild a -> ild a 
17 ild a  -> ld a s
18 ld a s -> d a sn
19 d a sn ->  a sno
20  a sno -> a snow
21 a snow ->  snowm
22  snowm -> snowma
23 snowma -> nowman
24 nowman -> owmanC
25 owmanC -> wmanCo
26 wmanCo -> manCom
27 manCom -> anCome
28 anCome -> nCome 
29 nCome  -> Come o
30 Come o -> ome on
31 ome on -> me on 
32 me on  -> e on l
33 e on l ->  on le
34  on le -> on let
35 on let -> n let'
36 n let' ->  let's
37  let's -> let's 
38 let's  -> et's g
39 et's g -> t's go
40 t's go -> 's go 
41 's go  -> s go a
42 s go a ->  go an
43  go an -> go and
44 go and -> o and 
45 o and  ->  and p
46  and p -> and pl
47 and pl -> nd pla
48 nd pla -> d play
49 d play ->  playI
50  playI 

In [33]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = np.eye(vocab_size)[x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [34]:
net = Net(vocab_size, hidden_size, 2)

In [35]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

torch.Size([116, 6, 25])
hahaaaaaaaaaaaaaaaahaahaaaaahaahaahaaaaaaaahaahaaahaaaaaaaaaaahaahhaaaaaaaahaahaaaahaaaaaaahaaaaaaaahahhaaaaaaaaaaaaaaaaa
oalaaaaaaa aaaaaaaaaaaaaaaaaaaaaaa aaaaa aaaaaaaaaa aaaaaaaaaaaaa  aaaaaaaaaaa aaaaaaaaa aaaaaaaa aaaaaaaaaaaaaaaaaaaaaaa
oe                                                                                                                       
oo                                                                                                                       
oo                                                                                                                       
o                                                                                                                        
o                                                                                                                        
                                                                                                                         

In [36]:
predict_str

"o you want to build a snowmanCome ou let's go and playI never see you anymore Come out the doorIt's like you've gone away"